<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Day-and-Month" data-toc-modified-id="Day-and-Month-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Day and Month</a></span></li><li><span><a href="#correlation" data-toc-modified-id="correlation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>correlation</a></span></li><li><span><a href="#Pair-Plot" data-toc-modified-id="Pair-Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pair Plot</a></span></li><li><span><a href="#skewd" data-toc-modified-id="skewd-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>skewd</a></span></li><li><span><a href="#Split-the-Data" data-toc-modified-id="Split-the-Data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Split the Data</a></span></li><li><span><a href="#5--Features-selection-useing-mutual-information" data-toc-modified-id="5--Features-selection-useing-mutual-information-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>5  Features selection useing mutual information</a></span></li></ul></div>

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.model_selection import train_test_split ,KFold, cross_val_score
from sklearn.linear_model import Lasso ,Ridge,LinearRegression
from sklearn.preprocessing import StandardScaler ,OneHotEncoder
from sklearn.metrics import mean_squared_error,auc, mean_squared_log_error
import os

from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso ,Ridge,LinearRegression
from sklearn.ensemble import GradientBoostingRegressor , RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import norm 

pd.set_option('display.max_columns', 80)
pd.set_option("max_r", 80)
pd.set_option('display.max_rows', 80)
%matplotlib inline

In [ ]:

data = pd.read_csv('../input/bike-sharing-demand/train.csv')
test = pd.read_csv('../input/bike-sharing-demand/test.csv')

data.drop(['casual','registered'],axis = 1 ,inplace=  True)
print('Train data shape:',data.shape)
print('Test data shape:',test.shape)
display(data.head(5))
print('Train data statsics:')
display(data.describe())

In [ ]:
print(data.info())
print(test.info())

# Day and Month 

In [ ]:
data['H'] =(data["workingday"]-1).abs() + data['holiday']
test['H'] =(test["workingday"]-1).abs() + test['holiday']

data['datetime'] = pd.to_datetime(data['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

data['month'] = data['datetime'].dt.month
data['Hour'] = data['datetime'].dt.hour

test['month'] = test['datetime'].dt.month
test['Hour'] = test['datetime'].dt.hour

data['WeekDay']=data['datetime'].dt.day_name()
test['WeekDay']=test['datetime'].dt.day_name()

data=pd.get_dummies(data,prefix="WD_",columns =['WeekDay'] )
test=pd.get_dummies(test,prefix="WD_", columns =['WeekDay'] )

data.drop(['datetime'],axis = 1 ,inplace=  True)


In [ ]:
data

In [ ]:
sns.histplot(data['count'])

fig, axes = plt.subplots(data.drop('count',axis= 1).select_dtypes(exclude ='object').shape[1], 1, figsize=(10, 42))
for idx,val in enumerate(data.drop('count',axis= 1).select_dtypes(exclude ='object').columns):
    sns.histplot(data[val],ax =axes[idx] , color='red' ,alpha= .3,element='step',kde=True,stat='probability',legend=True)
    sns.histplot(test[val],ax =axes[idx] , color='blue',alpha= .3 ,element='step',kde=True,stat='probability',legend=True)
    axes[idx].legend(labels=['Train', 'Test'])
plt.show()

# correlation

In [ ]:
plt.figure(figsize = (15,10))
corr = data.corr()
sns.heatmap(np.abs(corr), mask= np.triu(np.ones_like(corr, dtype=bool))
            ,center=0, square=True, linewidths=.5,annot=True)
plt.show()

# Pair Plot

In [ ]:
sns.pairplot(data , y_vars='count',aspect=1)


In [ ]:
data.columns

In [ ]:
sns.pairplot(data[['temp','atemp','humidity', 'windspeed',"workingday"]],hue="workingday", corner=True ,aspect=1)


# skewd 

In [ ]:
data.columns[abs( data.skew(axis= 0) ) > 0.5 ]

In [ ]:
from scipy.stats import boxcox 

data['Wind_speed_log'] ,lam=boxcox(data['windspeed'].values.reshape(-1)+1 )
test['Wind_speed_log'] = boxcox(test['windspeed'].values.reshape(-1)+1,lam)

fig, axes = plt.subplots(1, 1, figsize=(15, 6))
sns.histplot(data['Wind_speed_log'],ax =axes,color='red' ,alpha= .8,kde=True,stat='probability')
sns.histplot(test['Wind_speed_log'],ax =axes ,color='blue',alpha= .2 ,kde=True,stat='probability')

# Split the Data

In [ ]:
data['count']=np.log1p(data['count'])
xtrain ,xval,ytrain,yval =train_test_split(data.drop('count',axis=1),data['count'],
                                           test_size=.25, random_state=0, shuffle=True)

print(xtrain.shape ,xval.shape,ytrain.shape,yval.shape)

# 5  Features selection useing mutual information

In [ ]:
mi = mutual_info_regression(xtrain[['holiday','workingday','H']], ytrain, random_state=0).round(3)
# Print the name and mutual information score of each feature
for feature in zip(['holiday','workingday','H'], mi):
    print(feature)
print('-------------------------------------------------------------')    
mi = mutual_info_regression(xtrain[['WD__Friday','WD__Saturday', 'WD__Sunday', 'WD__Thursday', 'WD__Tuesday',
       'WD__Wednesday']], ytrain, random_state=0).round(3)
# Print the name and mutual information score of each feature
for feature in zip(['WD__Friday','WD__Saturday', 'WD__Sunday', 'WD__Thursday', 'WD__Tuesday',
       'WD__Wednesday'], mi):
    print(feature)    
print('-------------------------------------------------------------') 
mi = mutual_info_regression(xtrain[['temp', 'atemp']], ytrain, random_state=0).round(3)
# Print the name and mutual information score of each feature
for feature in zip(['temp', 'atemp'], mi):
    print(feature)  
print('-------------------------------------------------------------') 
mi = mutual_info_regression(xtrain[['Wind_speed_log', 'windspeed']], ytrain, random_state=0).round(3)
# Print the name and mutual information score of each feature
for feature in zip(['Wind_speed_log', 'windspeed'], mi):
    print(feature) 
print('-------------------------------------------------------------') 
mi = mutual_info_regression(xtrain, ytrain, random_state=0).round(3)
# Print the name and mutual information score of each feature
for feature in zip(xtrain.columns, mi):
    print(feature)  

In [ ]:
xtrain.columns

In [ ]:
dropcols=['atemp','holiday', 'WD__Sunday', 
       'WD__Tuesday', 'WD__Wednesday', 'Wind_speed_log']
data.drop(dropcols,axis = 1 ,inplace=  True)
xtrain.drop(dropcols,axis = 1 ,inplace=  True)
xval.drop(dropcols,axis = 1 ,inplace=  True)
test.drop(dropcols,axis = 1 ,inplace=  True)

In [ ]:

from sklearn.ensemble import ExtraTreesRegressor

et = ExtraTreesRegressor( n_estimators=150, max_depth=50,criterion='mse')

et.fit(xtrain ,ytrain)

#print(regressor.feature_importances_)

predt= et.predict(xtrain)
predv= et.predict(xval)

print("RMSE t:", np.sqrt(mean_squared_error(ytrain,predt )))
print("RMSE val:", np.sqrt(mean_squared_error(yval,predv )))

#print("RMSE val:", np.sqrt(mean_squared_error(yval,predv )))
#print("RMSLE val:", np.sqrt(mean_squared_log_error(yval+1,predv+1 )))


In [ ]:
perdtest = et.predict(test.drop('datetime',axis = 1) )
sample_submission= pd.DataFrame({'datetime':test['datetime'].to_numpy(), 'count':perdtest })
print(sample_submission.shape,test.shape)
sample_submission.to_csv(os.path.join('./',"submission.csv"), index=False)